# Welcome to the PyLibSIMBA walkthrough


We will go through the simple steps it takes to submit a message to a blockchain, and verify the message arrived

## First lets import pylibsimba

This is already installed in the notebook so we can do

In [1]:
import pylibsimba

## Next we need a Wallet

This holds the public and private keys used to sign a transaction, and creates an address to send the transactions from.

We will import a helper class and generate a wallet.

In [2]:
from pylibsimba.wallet import Wallet


We can generate a wallet to use on the SIMBAChain test blockchain.

In [3]:
wallet = Wallet(None)
wallet.generate_wallet('test1234')
addr = wallet.get_address()
print(addr)

0xc341e8e093f44d9149e20f39317da93def0dd02b


## Import a few things

Specifically we need the ability to get a SIMBAChain object.

Most of the outputs are in JSON format, so we import that too.

In [4]:
from pylibsimba import get_simba_instance
import json

Get the SIMBAChain object. This requires a few parameters

* url : The API URL
	* We'll use the test from simbachain.com
* wallet : The Wallet to use
	* This is the wallet we just made above
* api_key : (Optional) The API key
	* This is a test API pre generated for this example.
	* Please register at simbachain.com to gain the ability to create your own contracts and access them via you own 
	API key.
* management_key : (Optional) The Management API key
	* As for the api_key, but unused for now.
        

In [ ]:
simba = get_simba_instance(
    'https://api.simbachain.com/v1/libSimba-SimbaChat-Quorum/',
    wallet,
    '04d1729f7144873851a745d2ae85639f55c8e3de5aea626a2bcd0055c01ba6fc',
    '')

Lets check the balance of our wallet. To do this we simply get_balance() from the simba instance we've just created.

The output is a dict, so we'll use json to dump it to a string.

In [ ]:
balance = simba.get_balance()
print("Balance: {}".format(json.dumps(balance)))

## Calling a method on a Smart Contract on SIMBAChain.com

First, we need the parameters to call the method with.

As it's an example method, the params are given below. (We'll look at how to get the paramters for a different method later)

In [ ]:
method_params = {
    'assetId': "0xbad65ff688a28efdd17d979c12f0ab2e2de305dbc8a2aa6be45ed644da822cfb",
    'name': "A Test Room",
    'createdBy': "PyLibSIMBA",
}

Call the method with the parameters.

That's it!


In [ ]:
resp = simba.call_method('createRoom', method_params)
print("Successful submitting? {}".format(resp))

## Did it work?

There is a delay between submitting a transaction, and it appearing on the blockchain.

We have the option to "send and forget", or check to see if it has "deployed" successfully. 

To be completely sure, we can run wait_for_success_or_error() to check. 

Be aware this can take some time, and a variety of errors can be thrown if a problem is detected. Please check the documentation on Exceptions to learn about the types of errors caught by the PyLibSIMBA SDK.

In [ ]:
try:
    final_resp = simba.wait_for_success_or_error(resp)
    print("Successful? {}".format(final_resp))

except Exception as e1:
    print("Failure! {}".format(e1))

## Test 4 - Calling a method and submitting files

Similar to above, we need some method parameters, and we'll add some files too.

In [ ]:
method_params = {
	'assetId': "A Test Room",
	'chatRoom': "A Test Room",
	'message': "Hello World",
	'sentBy': "PyLibSIMBA"
}

files = ["test_files/test file 1.txt",
		 "test_files/test file 2.txt"]

Instead of call_method() we use call_method_with_file()

To be sure it worked, we'll wait again.

In [ ]:
resp = simba.call_method_with_file('sendMessage', method_params, files)
print("Successful submitting? {}".format(resp))

resp = simba.wait_for_success_or_error(resp)
print("Successfully deployed? {}".format(resp))